Scraping from Spotify: https://github.com/rupert/pandora-to-google-music/blob/master/pandora_to_google_music.py

In [1]:
import re
from getpass import getpass
from collections import defaultdict
import difflib

import requests
from termcolor import colored
from lxml import html
# import unidecode

In [2]:
#Import Usernames and PWs
from config import pan_user, pan_pw, spot_user, spot_pw

In [3]:
class LoginException(Exception):
    pass

In [4]:
class PandoraClient(object):
    LOGIN_URL = "https://www.pandora.com/login.vm"
    LIKES_URL = "http://www.pandora.com/content/tracklikes"
    STATIONS_URL = "http://www.pandora.com/content/stations"

In [5]:
def __init__(self, email, password):
        self.session = requests.session()

        response = self.session.post(PandoraClient.LOGIN_URL, data={
            "login_username": email,
            "login_password": password,
        })

        if "0;url=http://www.pandora.com/people/" not in response.text:
            raise LoginException("Pandora login failed, check email and password")

In [6]:
def liked_tracks(self):
#         """ Scrape likes from the Pandora web interface """

        like_start_index = 0
        thumb_start_index = 0

        tracks = defaultdict(list)
        more_pages = True
        page = 1

        while more_pages:
            response = self.session.get(PandoraClient.LIKES_URL, params={
                "likeStartIndex": like_start_index,
                "thumbStartIndex": thumb_start_index,
            })

            print_section_heading('Fetching Pandora Likes (page %d)' % page)

            tree = html.fromstring(response.text)

            for element in tree.find_class("infobox-body"):
                title = unicode(element.find("h3").text_content())
                title = title.strip()

                artist = unicode(element.find("p").text_content())
                artist = artist.strip()
                artist = re.sub(r"^by\s+", "", artist)

                station_elements = element.find_class("like_context_stationname")

                if station_elements:
                    station_name = unicode(station_elements[0].text_content())
                    station_name = station_name.strip()
                else:
                    # Bookmarked track
                    station_name = None

                tracks[station_name].append((artist, title))

                print_song(artist, title)

            more_elements = tree.find_class("show_more")

            # There are more pages
            if more_elements:
                like_start_index = more_elements[0].get("data-nextlikestartindex")
                thumb_start_index = more_elements[0].get("data-nextthumbstartindex")
            else:
                more_pages = False

            page += 1

        return tracks

In [7]:
def stations(self):
#         """ Scrape station names from the Pandora web interface """

        response = self.session.get(PandoraClient.STATIONS_URL)
        tree = html.fromstring(response.text)

        stations = []

        for element in tree.findall(".//h3"):
            station_name = unicode(element.text_content().strip())
            stations.append(station_name)

        return stations

In [8]:
def pandora_to_google_music(pan_user, pan_pw):
#     """ Sync Pandora likes with Google Music playlists """

    pandora_client = PandoraClient(pandora_email, pandora_password)

    # Get liked Pandora tracks
    pandora_likes = pandora_client.liked_tracks()
    pandora_like_count = sum(len(x) for x in pandora_likes.values())

    # Get Pandora stations
    pandora_stations = set(pandora_client.stations())

    pandora_playlists = defaultdict(list)

    # Copy all songs to main playlist
    # Add Pandora prefix to playlist names
    # Remove deleted stations (songs will be in main playlist)
    for station_name, songs in pandora_likes.items():
        # Copy songs to main playlist
        pandora_playlists["Pandora"].extend(songs)

        # Check station hasn't been deleted
        if station_name in pandora_stations:
            pandora_playlists["Pandora - %s" % station_name] = songs

In [9]:
def main():
#     """ Run pandora_to_google_music """
    pandora_to_google_music(
        pan_user,
        pan_pw
    )

<function __main__.main()>